## Libraries (local machine)

In [5]:
from dotenv import load_dotenv
from os import getenv
import numpy as np
import pandas as pd

load_dotenv()

False

## Libraries (Google Colab)

In [1]:
from google.colab import userdata
import numpy as np
import pandas as pd

def getenv(secretName: str, default_value):
  try:
    return userdata.get(secretName)
  except:
    return default_value

## Classes definition

In [24]:
#from typing import Any, NewType

class Supplier:
    def __init__(self, name: str, parameters: dict[str, list[float]]):
        self.name = name

        self.µ_price_high_complexity = getenv("AVG_PRICE_LOW_COMPLEXITY")
        self.σ_price_high_complexity = getenv("STDEV_PRICE_LOW_COMPLEXITY")
        self.µ_price_medium_complexity = getenv("AVG_PRICE_MEDIUM_COMPLEXITY")
        self.σ_price_medium_complexity = getenv("STDEV_PRICE_MEDIUM_COMPLEXITY")
        self.µ_price_low_complexity = getenv("AVG_PRICE_HIGH_COMPLEXITY")
        self.σ_price_low_complexity = getenv("STDEV_PRICE_HIGH_COMPLEXITY")

    def __str__(self):
        return self.name

class Part_Number:
    def __init__(self, pn: str, complexity: str):
        self.pn = pn
        self.complexity = complexity

    def __str__(self):
        return self.pn

class ECN:
    def __init__(self, ecn_id: str, pn_list: list[Part_Number]):
        self.ecn_id = ecn_id
        self.items = pn_list

    def __str__(self):
        return self.ecn_id

class RFQ:
    def __init__(self, ecn: ECN):
        self.ecn = ecn

In [33]:
class Environment:
  def __init__(self):
    self.suppliers = []
    self.rfqs = []
    self.ecns = []
    self.part_kinds = {
        "A": {
            "average": float(getenv("AVG_A_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_A_PART_KIND", 1)),
            "complexity": {"low": 0.6818181818182, "medium": 0.318181818181818, "high": 0},
            "parts": []
        },
        "B": {
            "average": float(getenv("AVG_B_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_B_PART_KIND", 1)),
            "complexity": {"low": 1/3, "medium": 2/3, "high": 0},
            "parts": []
        },
        "C": {
            "average": float(getenv("AVG_C_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_C_PART_KIND", 1)),
            "complexity": {"low": 1, "medium": 0, "high": 0},
            "parts": []
        },
        "D": {
            "average": float(getenv("AVG_D_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_D_PART_KIND", 1)),
            "complexity": {"low": 0.090909090909090909, "medium": 0.727272727272727, "high": 0.181818181818182},
            "parts": []
        },
        "E": {
            "average": float(getenv("AVG_E_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_E_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        },
        "F": {
            "average": float(getenv("AVG_F_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_F_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        },
        "G": {
            "average": float(getenv("AVG_G_PART_KIND", 0)),
            "stdev": float(getenv("STDEV_G_PART_KIND", 1)),
            "complexity": {"low": 0, "medium": 0, "high": 1},
            "parts": []
        }
    }

  def gen_ecn(self, qty: int):
    for i in range(qty):
      ecn_part_numbers = []

      while len(ecn_part_numbers) == 0:
        for key in self.part_kinds.keys():
          kind_complexity_keys = list(self.part_kinds[key]["complexity"].keys())
          kind_complexity_probabilities = list(self.part_kinds[key]["complexity"].values())

          for j in range(max(int(np.random.normal(self.part_kinds[key]["average"], self.part_kinds[key]["stdev"])), 0)):
            category_part_number = len(self.part_kinds[key]["parts"]) + 1
            complexity = np.random.choice(kind_complexity_keys, p=kind_complexity_probabilities)

            part_number = Part_Number(pn=f"A0{key}{str(category_part_number).zfill(6)}", complexity=complexity)

            self.part_kinds[key]["parts"].append(part_number)
            ecn_part_numbers.append(part_number)

      ecn_number = len(self.ecns) + 1
      self.ecns.append(ECN(ecn_id=f"ECN{str(ecn_number).zfill(7)}", pn_list=ecn_part_numbers))

In [34]:
env = Environment()
env.gen_ecn(10)